In [367]:
# 라이브러리 import
import pandas as pd
import numpy as np

from selenium import webdriver  # 라이브러리(모듈) 가져오라
from selenium.webdriver import ActionChains as AC
import chromedriver_autoinstaller
from tqdm import tqdm
from tqdm.notebook import tqdm
import re
from time import sleep
import time
from tqdm import tqdm_notebook

from konlpy.tag import Hannanum
import requests
from bs4 import BeautifulSoup
from konlpy.tag import Twitter
from collections import Counter
# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

# 추천시스템
import surprise
from surprise import SVD
from surprise import NMF
from surprise import KNNBasic

In [2]:
# 데이터 수집할 키워드 지정
keyword = "고기"
keyword

'고기'

In [3]:
# 크롬창 띄우기
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)

driver.get("https://www.siksinhot.com/search?keywords={}".format(keyword))
time.sleep(2)

In [4]:
# 리뷰 버튼 클릭
driver.find_element_by_css_selector("#contents > div.sub_contens > div > div > div.word_sch_box > div > div > ul > li:nth-child(4) > a").click( )
time.sleep(1)

In [20]:
"""
#더보기 버튼 누르기
driver.find_element_by_css_selector("#schMove4 > div > a > span").click( )
time.sleep(1)
"""

In [8]:
#더보기 버튼 여러번 누르기


while True:  # 더보기 횟수만큼 반복 
    try:
        driver.find_element_by_css_selector("#schMove4 > div > a > span").click( )
        time.sleep(1)
    except:        
        break   # 에러나면 클릭 반복문을 빠져나가라


In [16]:
"""
#내용 정리
nickname = driver.find_element_by_css_selector('div > div > div.name_data').text
score = driver.find_element_by_css_selector('div > div > div.score_story > div').text
story = driver.find_element_by_css_selector('div > div > div.score_story > p').text
name = driver.find_element_by_css_selector('div > div > div.store_area_menu > strong').text
area = driver.find_element_by_css_selector(' div > div > div.store_area_menu > span').text
"""


In [9]:
# area 수집하기
area_list = []

areas = driver.find_elements_by_css_selector('div > div > div.store_area_menu > span') 

for area in tqdm(areas):    
    area = area.text
    area_list.append(area)
    
    #schMove4 > div > ul 
    #div > div > div.score_story > p
    #schMove4 > div > ul > div > div > ul
    
area_list

['충북, 단양',
 '충북, 단양',
 '충남, 아산시/아산배방',
 '서울-강남, 논현/학동',
 '전남, 여수시',
 '강원, 춘천시',
 '경북, 칠곡',
 '전남, 여수시',
 '경남, 양산',
 '경기, 이천시',
 '강원, 춘천시',
 '전남, 여수시',
 '경남, 거제시',
 '경남, 김해',
 '충남, 세종시/조치원',
 '대구, 북구/경북대/칠곡',
 '부산, 해운대',
 '전남, 목포',
 '충북, 단양',
 '충남, 예산',
 '충남, 예산',
 '강원, 춘천시',
 '전남, 목포',
 '전남, 목포',
 '경남, 합천',
 '경남, 김해',
 '경북, 군위',
 '경기, 부천시',
 '전남, 목포',
 '서울-강남, 구로/대림/가산/금천구',
 '경북, 안동',
 '서울-강남, 강남역',
 '서울-강남, 압구정',
 '경북, 예천',
 '대구, 북구/경북대/칠곡',
 '경기, 파주시',
 '서울-강북, 월곡/석계',
 '대구, 북구/경북대/칠곡',
 '대구, 북구/경북대/칠곡',
 '경북, 군위',
 '대구, 남구/봉덕/대명',
 '대구, 북구/경북대/칠곡',
 '충북, 음성',
 '서울-강남, 논현/학동',
 '대구, 북구/경북대/칠곡',
 '경기, 화성-동탄신도시',
 '대구, 북구/경북대/칠곡',
 '서울-강남, 논현/학동',
 '서울-강북, 인사동',
 '인천, 부평',
 '서울-강남, 당산역',
 '강원, 태백',
 '경북, 포항',
 '경북, 안동',
 '경기, 화성-동탄신도시',
 '대전, 유성-상대',
 '경남, 진주시',
 '대전, 유성-상대',
 '경기, 부천시',
 '대전, 유성-상대',
 '경기, 용인-보정',
 '충남, 공주시',
 '울산, 울주군',
 '강원, 태백',
 '경남, 진주시',
 '경남, 진주시',
 '제주, 제주시내',
 '경기, 구리',
 '경기, 가평군',
 '경남, 거창',
 '경북, 울진',
 '경남, 진주시',
 '경남, 함안',
 '경기, 남양주/덕소/화도',
 '광주, 남구/봉선/주월'

In [10]:
# name 수집하기
name_list = []

names = driver.find_elements_by_css_selector('div > div > div.store_area_menu > strong') 

for name in tqdm(names):    
    name = name.text
    name_list.append(name)
    
    #schMove4 > div > ul 
    #div > div > div.score_story > p
    #schMove4 > div > ul > div > div > ul
    
name_list

['소백한우',
 '소백한우',
 '잭아저씨족발보쌈',
 '강남고기집 신칠성집',
 '압구정',
 '닭발어때',
 '칠일곱창',
 '청운갈비',
 '도야족발',
 '오동추야',
 '닭발어때',
 '청운갈비',
 '서씨네삼겹살',
 '부영보쌈',
 '청사한우갤러리',
 '오븐에구운삼겹살과 오븐에구운 치킨',
 '수부칸',
 '곰집갈비',
 '소백한우',
 '고덕갈비',
 '고덕갈비',
 '닭발어때',
 '송월갈비',
 '송월갈비',
 '한우리가든',
 '부영보쌈',
 '민속한우갈비',
 '개성보쌈족발',
 '송월갈비',
 '마포집한우곱창',
 '구서울갈비',
 '호미불닭발',
 '근식당',
 '오르비',
 '오븐에구운삼겹살과 오븐에구운 치킨',
 '28치킨',
 '천궁족발보쌈',
 '오븐에구운삼겹살과 오븐에구운 치킨',
 '오븐에구운삼겹살과 오븐에구운 치킨',
 '민속한우갈비',
 '효명통닭',
 '오븐에구운삼겹살과 오븐에구운 치킨',
 '고당다한우마을',
 '왕소금구이',
 '오븐에구운삼겹살과 오븐에구운 치킨',
 '갈풍집',
 '오븐에구운삼겹살과 오븐에구운 치킨',
 '강남고기집 신칠성집',
 '해몽',
 '배가족발',
 '이필당 왕족발보쌈 본점',
 '부일갈비',
 '만포한우갈비',
 '안동명성한우갈비',
 '갈풍집',
 '어화장단',
 '홍제원',
 '궁중보쌈',
 '파절이삼겹살',
 '궁중보쌈',
 '보정족발',
 '명성불고기',
 '계명숯불고기',
 '할매곱창',
 '감탄고기집',
 '감탄고기집',
 '신마포갈매기',
 '보배곱창',
 '소문난닭갈비',
 '명덕총각족발',
 '송학면옥',
 '감탄고기집',
 '본가삼계탕',
 '한성불고기',
 '남매보쌈',
 '수부칸',
 '개성보쌈족발',
 '송월갈비',
 '동가래한우셀프점',
 '본가삼계탕',
 '보배곱창',
 '군위이로운한우',
 '궁중보쌈',
 '향남항아리보쌈',
 '파절이삼겹살',
 '개성보쌈족발',
 '전복오리누룽지백숙',
 '홍림곱창',
 '송월갈비',
 '대숯맛사랑',
 '수부칸',
 '

In [11]:
# score 수집하기
point_list = []

points = driver.find_elements_by_css_selector('div > div > div.score_story > div') 

for point in tqdm(points):    
    point = point.text
    point_list.append(point)
    
    #schMove4 > div > ul 
    #div > div > div.score_story > p
    #schMove4 > div > ul > div > div > ul
    
point_list

['3.0',
 '5.0',
 '3.0',
 '5.0',
 '3.0',
 '2.5',
 '3.0',
 '3.0',
 '2.5',
 '3.0',
 '2.5',
 '3.0',
 '2.5',
 '3.0',
 '3.0',
 '3.0',
 '5.0',
 '3.5',
 '5.0',
 '3.0',
 '4.0',
 '3.0',
 '3.0',
 '5.0',
 '2.5',
 '3.0',
 '2.5',
 '3.0',
 '4.0',
 '3.0',
 '3.0',
 '3.0',
 '5.0',
 '3.0',
 '3.0',
 '3.0',
 '3.0',
 '3.0',
 '3.0',
 '2.5',
 '5.0',
 '3.0',
 '3.0',
 '2.5',
 '3.0',
 '3.0',
 '3.0',
 '5.0',
 '3.0',
 '4.0',
 '3.0',
 '5.0',
 '3.0',
 '3.0',
 '3.0',
 '3.0',
 '3.0',
 '5.0',
 '3.0',
 '3.0',
 '3.0',
 '5.0',
 '2.5',
 '3.0',
 '3.0',
 '2.5',
 '3.0',
 '5.0',
 '3.0',
 '3.0',
 '3.0',
 '2.5',
 '2.5',
 '3.0',
 '3.0',
 '5.0',
 '3.0',
 '3.0',
 '3.0',
 '3.0',
 '5.0',
 '4.0',
 '3.0',
 '3.0',
 '3.0',
 '3.0',
 '3.0',
 '3.0',
 '3.0',
 '5.0',
 '5.0',
 '3.0',
 '4.0',
 '5.0',
 '3.0',
 '5.0',
 '3.0',
 '3.0',
 '3.0',
 '3.0',
 '3.0',
 '3.0',
 '3.0',
 '2.5',
 '5.0',
 '2.5',
 '4.0',
 '3.0',
 '3.0',
 '4.0',
 '5.0',
 '3.0',
 '3.0',
 '3.0',
 '2.5',
 '3.0',
 '2.5',
 '3.0',
 '4.0',
 '3.0',
 '3.0',
 '3.0',
 '3.5',
 '3.0',
 '3.0',


In [12]:
# review 수집하기
review_list = []

reviews = driver.find_elements_by_css_selector("div > div > div.score_story > p") 

for review in tqdm(reviews):    
    review = review.text
    review_list.append(review)
    
    #schMove4 > div > ul 
    #div > div > div.score_story > p
    #schMove4 > div > ul > div > div > ul
    
review_list


['이집 고기 진짜 맛나요',
 '꽃등심 진짜 맛있네요',
 '요즘 영~족발 맛이 없어요',
 '한우가 이렇게 맛있는건지 이 집을 통해 알았네요 고기가 입안에서 살살 녹는다는게 어떤건지도요 추천하는집입니다!!!',
 '꽃등심 맛도좋아요',
 '닭발은 거의 익혀서 나와요',
 '나쁘진 않아요.',
 '삼겹살 맛있었었던 거 같구요',
 '족발과 보쌈 좋아하시는 분들은실패없을 곳',
 '수제갈비 넘나 맛있어요',
 '살이 통통한 닭발 ^^',
 '삼겹살 그런대로 맛도좋아요',
 '고기 육즙 살아있음',
 '보쌈은 언제나 부영보쌈이 진리다',
 '모듬한우 주문해서 먹었는데 갈비살이랑 채끝이 제일 괜찮았던듯 ? ㅋㅋ',
 '삼겹살 깔끔했음',
 '불고기를 푸짐하게 주는 집이다. 고기도 맛있다.',
 '수육무침 갈비 좋아요',
 '단양의 맛집입니다 꼭 한번 들러보세요',
 '한우갈비 좋다',
 '한우갈비 좋았던 것 같아요',
 '닭발말고도 막창이랑 삼겹살이랑 같이 먹을수 있는 곳',
 '소양념갈비 맛있다는데요.',
 '송월갈비 너무 맛있어용ㅎㅎ',
 '갈비살이 맛있다는데 삼겹살 먹었는데 맛있었어요~!~!',
 '맛있습니다',
 '한우는 정말 푸짐하게 먹을수 있는 곳',
 '언제나 훌륭한 족발 족발이 답이다',
 '소갈비랑 돼지갈비 맛있어요',
 '마포집한우곱창 신도림역점 당일 도축한 한우만 사용한답니다~',
 '지금까지 먹어 본 한우 갈비 중에 가장 맛있어요! 멀어서 자주 못 가겠지만 인생 한우 갈비집을 찾은 느낌 ㅜㅜ 먹는데 사라지는게 아까울 정도로 한우 갈비살이 정말 맛있어요!',
 '닭발 너무 맛있어여ㅋㅋㅋㅋ 소주룰 부르는 ㅋㅋㅋㅋ 닭발닭발닭발 닭발닭발닭발 닭발닭발닭발 닭발닭발닭발',
 '한우 투뿔을 이가격에.. 고기도 정말 최상급',
 '고기의 질이 비교가 안되게 조음요 한우불고기 추천!!',
 '좋은 맛이었던 거 같구요',
 "'28치킨' 집에 들려 아이들이 좋아하는 치킨 포장해왔네요. 치킨냄새가 아주 맛있게 나네요. *^^*",
 '보쌈도 잘 삶는곳 보쌈김치도

In [13]:
# nickname 수집하기
nickname_list = []

nicknames = driver.find_elements_by_css_selector("div > div > div.name_data") 

for a in tqdm(nicknames):    
    a = a.text
    nickname_list.append(a)
    
    #schMove4 > div > ul 
    #div > div > div.score_story > p
    #schMove4 > div > ul > div > div > ul
    #schMove4 > div > ul > li:nth-child(2) > div > div > div.score_story > p
    
nickname_list


['533464',
 '533635',
 '식신도라도라',
 'ROUTIS-K',
 '만두주떼요',
 '로또오빠',
 '맛집온니',
 '얼룩점박이',
 '대머리독수리',
 '과자는홈런볼',
 'ⅱ애기야ⅱ',
 '#Dali_Mom',
 '533301',
 '김라면이',
 '쉐잌잇><',
 '니가제일예뻤어',
 '360591',
 '보아라돌이',
 '534398',
 '마돈나스타킹',
 '마약김밥중독자',
 '촤컬릿퐁듀우♡',
 '슈가예스플리즈',
 '454745',
 '이게내닉네임',
 '여기어때0925',
 '장원wanga',
 '533320',
 '454744',
 '학교종',
 '알람몬1313',
 '29',
 '450824',
 '여자지구인',
 '헬로키티야',
 '호두까기인형',
 '맨날맥날',
 '잘생기면다오빠',
 '2차원개그',
 '식후식곤증',
 '453334',
 '나는귀염둥이',
 '새우깡버거',
 '막먹는영애씨',
 '힙통령',
 '533373',
 '킴민디띠',
 'AcRIte',
 '초콜렛덕후',
 '359487',
 '셔리0301',
 '핸섬맨',
 '무늬만공주',
 '카레맛호빵맨',
 '533068',
 'AlwaysAwake',
 '디제이퐉',
 '418190',
 '하얀고래',
 '나만꿀피부♡',
 '202동202호',
 '534352',
 '두부같은내얼굴',
 '띠드버거주세용',
 '달콤살벌한연애',
 '533111',
 '533295',
 '485020',
 '어마마마987',
 '손하트♡',
 'epi1122',
 '쿵떡♥찰떡',
 '오피스백설공주',
 '대추나무사람걸렸넹',
 '아이고아이고',
 '360591',
 '533324',
 '수포자',
 'ⅱ애기야ⅱ',
 '533273',
 '454472',
 '할_닉네임이_없다',
 '뜀박질여사',
 '호랑이기운이쑥쑥',
 '미쯔랑우유',
 '533199',
 '분홍꼬까신',
 '난P양이라고해',
 '촤컬릿퐁듀우♡',
 '534491',
 '43016',
 '2차원개그',
 '

In [16]:
df2 = pd.DataFrame({"닉네임":nickname_list,
                    "평가":review_list,
                    "평점":point_list,
                   "가게명":name_list,
                   "지역":area_list})

df2.to_excel("siksin.xlsx", encoding='utf-8-sig')

In [101]:

# 데이터 수집할 키워드 지정
keyword = "고기"
keyword


# 크롬창 띄우기
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)

driver.get("https://www.siksinhot.com/search?keywords={}".format(keyword))
time.sleep(2)


# 리뷰 버튼 클릭
driver.find_element_by_css_selector("#contents > div.sub_contens > div > div > div.word_sch_box > div > div > ul > li:nth-child(4) > a").click( )
time.sleep(1)





nickname = driver.find_element_by_css_selector('#schMove4 > div > ul > li:nth-child(1) > div > div > div.name_data').text
score = driver.find_element_by_css_selector('#schMove4 > div > ul > li:nth-child(1) > div > div > div.score_story > div').text
review = driver.find_element_by_css_selector('#schMove4 > div > ul > li:nth-child(1) > div > div > div.score_story > p').text
name = driver.find_element_by_css_selector('#schMove4 > div > ul > li:nth-child(1) > div > div > div.store_area_menu > strong').text
area = driver.find_element_by_css_selector('#schMove4 > div > ul > li:nth-child(1) > div > div > div.store_area_menu > span').text

for i in range(0,10):
    driver.find_element_by_css_selector("#schMove4 > div > a > span").click( )

"""
# 더보기 버튼 여러번 클릭하기
while True:  # 더보기 횟수만큼 반복 
    try:
        driver.find_element_by_css_selector("#schMove4 > div > a > span").click( )
        time.sleep(1)
    except:        
        break   # 에러나면 클릭 반복문을 빠져나가라
"""
# review 수집하기
review_list = []

reviews = driver.find_elements_by_css_selector("#schMove4 > div")

for review in tqdm(reviews):    
    review = review.text
    review_list.append(review)       
            
        
target_info = {}

target_info['닉네임'] = nickname
target_info['평점'] = score
target_info['댓글내용'] = review
target_info['상호명'] = name
target_info['지역'] = area

dict[i] = target_info
        

dict



{9: {'닉네임': '533635',
  '평점': '5.0',
  '댓글내용': '533635\n5.0\n꽃등심 진짜 맛있네요\n소백한우\n충북, 단양\n고기/구이류\n533464\n3.0\n이집 고기 진짜 맛나요\n소백한우\n충북, 단양\n고기/구이류\n식신도라도라\n3.0\n요즘 영~족발 맛이 없어요\n잭아저씨족발보쌈\n충남, 아산시/아산배방\n고기/구이류\nROUTIS-K\n5.0\n한우가 이렇게 맛있는건지 이 집을 통해 알았네요 고기가 입안에서 살살 녹는다는게 어떤건지도요 추천하는집입니다!!!\n갈비살\n한우\n고기집\n차돌\n강남고기집 신칠성집\n서울-강남, 논현/학동\n고기/구이류\n더보기',
  '상호명': '소백한우',
  '지역': '충북, 단양'}}

In [172]:
df2

,닉네임,평가,평점,가게명,지역
0,533464,이집 고기 진짜 맛나요,3.0,소백한우,"충북, 단양"
1,533635,꽃등심 진짜 맛있네요,5.0,소백한우,"충북, 단양"
2,식신도라도라,요즘 영~족발 맛이 없어요,3.0,잭아저씨족발보쌈,"충남, 아산시/아산배방"
3,ROUTIS-K,한우가 이렇게 맛있는건지 이 집을 통해 알았네요 고기가 입안에서 살살 녹는다는게 어...,5.0,강남고기집 신칠성집,"서울-강남, 논현/학동"
4,만두주떼요,꽃등심 맛도좋아요,3.0,압구정,"전남, 여수시"
...,...,...,...,...,...
6408,캔디크러시소다,자기 전에 생각나는 맛,3.0,대전식당,"경남, 거창"
6409,엽떡매냐,추천할만한 편이에요,3.0,진가네대구갈비,"경북, 경주시"
6410,Eunvely:3,냉면 깔끔하구요,3.0,혼섬갈비,"제주, 제주시내"
6411,441023,재방문 의사 있음.,4.0,점례네,"부산, 해운대"


In [173]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6413 entries, 0 to 6412
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   닉네임     6413 non-null   object
 1   평가      6413 non-null   object
 2   평점      6413 non-null   object
 3   가게명     6413 non-null   object
 4   지역      6413 non-null   object
dtypes: object(5)
memory usage: 250.6+ KB


In [174]:
df2.duplicated('닉네임')

0       False
1       False
2       False
3       False
4       False
        ...  
6408     True
6409     True
6410     True
6411     True
6412     True
Length: 6413, dtype: bool

In [330]:
data = pd.read_excel('siksin_2.xlsx')

In [331]:
data

,Unnamed: 0,닉네임,평가,평점,가게명,지역
0,273,!날아라슈퍼맨!,매운족발 그냥 깔끔합니다.,3.0,꿀벅지족발,"서울-강북, 남산/후암동"
1,976,!날아라슈퍼맨!,한우 갈비살 양념구이 먹엇는데 석쇠에 빨리빨리 구워서 먹는데 여기특이한게 고추장에 ...,4.0,영양숯불갈비,"경북, 경주시"
2,2438,!날아라슈퍼맨!,닭불고기 정말 만족해요,3.0,무등산닭불고기,"광주, 서구-금호/풍암"
3,3341,!날아라슈퍼맨!,제주도에서 먹은 고기국수 만큼 괜찮았어요 ㅎㅎ,3.0,아랑졸디,"경기, 구리"
4,3467,!날아라슈퍼맨!,누룽지백숙!! 백숙국물에 누룽지를 넣어서 나오는데 담백하고 고소하고 닭백숙이 되게 ...,3.0,산촌,"경기, 의왕/백운호수"
...,...,...,...,...,...,...
5340,6024,힙통령,만족스러워요,2.5,내림손삼계탕,"서울-강북, 명동"
5341,628,힙합간지녀,돼지갈비도 인기 많아요,3.0,청석정육식당,"경기, 광주시"
5342,767,힙합간지녀,등심 진짜 맛있어요,3.0,창신식육식당,"경북, 영주"
5343,2214,힙합간지녀,고기를 직접 골라서 먹을 수 있는게 정말 맘에 들어요 ㅋㅋ 한우랑 키조개랑 같이 먹...,3.0,탐마루장흥한우,"전남, 장흥"


In [332]:
print(data.shape)

(5345, 6)


In [336]:
df = data[['닉네임', '가게명', '평점']]
df['평점'] = df['평점'].astype(int)
df

,닉네임,가게명,평점
0,!날아라슈퍼맨!,꿀벅지족발,3
1,!날아라슈퍼맨!,영양숯불갈비,4
2,!날아라슈퍼맨!,무등산닭불고기,3
3,!날아라슈퍼맨!,아랑졸디,3
4,!날아라슈퍼맨!,산촌,3
...,...,...,...
5340,힙통령,내림손삼계탕,2
5341,힙합간지녀,청석정육식당,3
5342,힙합간지녀,창신식육식당,3
5343,힙합간지녀,탐마루장흥한우,3


In [337]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5345 entries, 0 to 5344
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   닉네임     5345 non-null   object
 1   가게명     5345 non-null   object
 2   평점      5345 non-null   int32 
dtypes: int32(1), object(2)
memory usage: 104.5+ KB
